In [45]:
import http.client
import json

class ArpEntry:
    def __init__(self, interface, ip_address, mac_address, entry_type):
        self.interface = interface
        self.ip_address = ip_address
        self.mac_address = mac_address
        self.entry_type = entry_type
        self.vendor = self.__get_mac_vendor(self.mac_address)

    def __get_mac_vendor(self, mac_address):
        mac_address = mac_address.replace('-', ':')
        print(f'{mac_address=}')
        conn = http.client.HTTPSConnection("www.macvendorlookup.com")
        conn.request("GET", f"/api/v2/{mac_address}")
    
        response = conn.getresponse()
        data = response.read()
    
        if response.status == 200:
            try:
                return json.loads(data)[0]['company']
            except:
                return None
        else:
            return None    

In [46]:
help(ArpEntry)

Help on class ArpEntry in module __main__:

class ArpEntry(builtins.object)
 |  ArpEntry(interface, ip_address, mac_address, entry_type)
 |
 |  Methods defined here:
 |
 |  __init__(self, interface, ip_address, mac_address, entry_type)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



In [47]:
import re
def parse_arp_output(output):
    arp_entries = []
    current_interface = None

    for line in output.splitlines():
        interface_match = re.match(r'Interface: (\S+)', line)
        if interface_match:
            current_interface = interface_match.group(1)
        else:
            entry_match = re.match(r'\s+(\S+)\s+(\S+)\s+(\S+)', line)
            if entry_match and current_interface:
                ip_address, mac_address, entry_type = entry_match.groups()
                arp_entries.append(ArpEntry(current_interface, ip_address, mac_address, entry_type))

    return arp_entries


In [48]:
arp_output = """
Interface: 172.20.124.151 --- 0x5
  Internet Address      Physical Address      Type
  172.20.124.1          00-11-22-33-44-55     dynamic
  172.20.125.255        ff-ff-ff-ff-ff-ff     static
  224.0.0.22            01-00-5e-00-00-16     static
  224.0.0.251           01-00-5e-00-00-fb     static
  224.0.0.252           01-00-5e-00-00-fc     static
  239.255.255.250       01-00-5e-7f-ff-fa     static

Interface: 192.168.86.27 --- 0xe
  Internet Address      Physical Address      Type
  192.168.86.1          70-3a-cb-ab-9f-35     dynamic
  224.0.0.22            01-00-5e-00-00-16     static
  255.255.255.255       ff-ff-ff-ff-ff-ff     static
"""

arp_entries = parse_arp_output(arp_output)

mac_address='Address'
mac_address='00:11:22:33:44:55'
mac_address='ff:ff:ff:ff:ff:ff'
mac_address='01:00:5e:00:00:16'
mac_address='01:00:5e:00:00:fb'
mac_address='01:00:5e:00:00:fc'
mac_address='01:00:5e:7f:ff:fa'
mac_address='Address'
mac_address='70:3a:cb:ab:9f:35'
mac_address='01:00:5e:00:00:16'
mac_address='ff:ff:ff:ff:ff:ff'


In [49]:
for entry in arp_entries:
    print(f"Interface: {entry.interface}, IP Address: {entry.ip_address}, MAC Address: {entry.mac_address}, Type: {entry.entry_type} {entry.vendor} ")


Interface: 172.20.124.151, IP Address: Internet, MAC Address: Address, Type: Physical None 
Interface: 172.20.124.151, IP Address: 172.20.124.1, MAC Address: 00-11-22-33-44-55, Type: dynamic CIMSYS Inc 
Interface: 172.20.124.151, IP Address: 172.20.125.255, MAC Address: ff-ff-ff-ff-ff-ff, Type: static None 
Interface: 172.20.124.151, IP Address: 224.0.0.22, MAC Address: 01-00-5e-00-00-16, Type: static None 
Interface: 172.20.124.151, IP Address: 224.0.0.251, MAC Address: 01-00-5e-00-00-fb, Type: static None 
Interface: 172.20.124.151, IP Address: 224.0.0.252, MAC Address: 01-00-5e-00-00-fc, Type: static None 
Interface: 172.20.124.151, IP Address: 239.255.255.250, MAC Address: 01-00-5e-7f-ff-fa, Type: static None 
Interface: 192.168.86.27, IP Address: Internet, MAC Address: Address, Type: Physical None 
Interface: 192.168.86.27, IP Address: 192.168.86.1, MAC Address: 70-3a-cb-ab-9f-35, Type: dynamic Google, Inc. 
Interface: 192.168.86.27, IP Address: 224.0.0.22, MAC Address: 01-00-5e-0

In [50]:
first = arp_entries[0]
first.__dict__


{'interface': '172.20.124.151',
 'ip_address': 'Internet',
 'mac_address': 'Address',
 'entry_type': 'Physical',
 'vendor': None}